In [13]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Input
from tensorflow.math import confusion_matrix
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.math import confusion_matrix
from tensorflow.keras import layers
from tensorflow.data import Dataset
from tensorflow.keras.backend import clear_session
from tensorflow.data import AUTOTUNE
from tensorflow.keras.utils import image_dataset_from_directory

import numpy as np
import plotly.express as px

## Modeling the System

### Data Processing

In [14]:
def rescale() -> Sequential:

    rescale_layer = Sequential(layers.Rescaling(1./255))

    return rescale_layer

### Data Augmentation

In [15]:
def data_aug() -> Sequential:

    data_augmentation = Sequential(layers.RandomFlip("horizontal"))

    return data_augmentation

### Model Architecture

In [29]:
def architecture() -> Sequential:

    num_classes = 10

    model = Sequential()

    model.add(Conv2D(32, kernel_size = (3, 3), activation = "relu"))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Conv2D(32, kernel_size = (3, 3), activation = "relu"))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation = "relu"))
    model.add(Dense(128, activation = "relu"))
    model.add(Dense(128, activation = "relu"))
    model.add(Dense(num_classes, activation = "softmax"))

    return model

### Unifying the Model

In [37]:
def unify_model(architecture: Sequential) -> Sequential:
    
    input_shape   = (28, 28, 1)
    inputs        = Input(shape = input_shape)
    outputs       = architecture(inputs)
    model         = Model(inputs = inputs, outputs = outputs)

    return model

model = unify_model(architecture())

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 sequential_1 (Sequential)   (None, 10)                146410    
                                                                 
Total params: 146,410
Trainable params: 146,410
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def data_pipeline(self, directory: str, subset: str, rescale_layer: Sequential, augment_layer: Sequential) -> Dataset:

    batch_size = 128
    
    ds = image_dataset_from_directory(directory = directory,
                                      image_size = (28, 28),
                                      batch_size = batch_size,
                                      color_mode = "grayscale",
                                      subset = subset,
                                      seed = 1,
                                      validation_split = 0.2)
        

## Loading the data and compiling the model

In [36]:
clear_session()

rescale_layer = rescale()

train_ds = image_dataset_from_directory("C:/Users/Joao/Kotoamatsukami/Kotoamatsukami/data/fashion_mnist/train",
                                        image_size = (28, 28),
                                        batch_size = 128,
                                        color_mode = "grayscale",
                                        subset = "training",
                                        seed = 1,
                                        validation_split = 0.2)

train_ds = train_ds.map(lambda x, y: (rescale_layer(x), y), num_parallel_calls = AUTOTUNE)
train_ds = train_ds.shuffle(len(train_ds)).prefetch(buffer_size = AUTOTUNE).cache()

val_ds = image_dataset_from_directory("C:/Users/Joao/Kotoamatsukami/Kotoamatsukami/data/fashion_mnist/train",
                                        image_size = (28, 28),
                                        batch_size = 128,
                                        color_mode = "grayscale",
                                        subset = "validation",
                                        seed = 1,
                                        validation_split = 0.2)

val_ds = val_ds.map(lambda x, y: (rescale_layer(x), y), num_parallel_calls = AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size = AUTOTUNE).cache()


Found 60000 files belonging to 10 classes.
Using 48000 files for training.
Found 60000 files belonging to 10 classes.
Using 12000 files for validation.


In [ ]:
def confusion_matrix(model: Sequential, test_data):
    
    predictions = model.predict(test_data)
    preds = [np.argmax(p) for p in predictions]
    confusion = confusion_matrix(labels = test_targets, predictions = preds, num_classes = len(np.unique(test_targets)))
    fig = px.imshow(confusion, color_continuous_scale = "viridis")
    fig.show()

In [39]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

history = model.fit(train_ds,
                    epochs = 10,
                    verbose = 1,
                    validation_data = val_ds)

loss, acc = model.evaluate(val_ds)

# predictions = model.predict(test_data)
# preds = [np.argmax(p) for p in predictions]
# confusion = confusion_matrix(labels = test_targets, predictions = preds, num_classes = len(np.unique(test_targets)))
# fig = px.imshow(confusion, color_continuous_scale = "viridis")
# fig.show()

Epoch 1/10
375/375 [==============================] - 11s 27ms/step - loss: 0.1845 - accuracy: 0.9308 - val_loss: 0.2689 - val_accuracy: 0.9038
Epoch 2/10
375/375 [==============================] - 12s 32ms/step - loss: 0.1717 - accuracy: 0.9359 - val_loss: 0.2877 - val_accuracy: 0.8972
Epoch 3/10
375/375 [==============================] - 13s 34ms/step - loss: 0.1599 - accuracy: 0.9403 - val_loss: 0.2846 - val_accuracy: 0.9018
Epoch 4/10
375/375 [==============================] - 12s 31ms/step - loss: 0.1499 - accuracy: 0.9440 - val_loss: 0.2945 - val_accuracy: 0.8995
Epoch 5/10
375/375 [==============================] - 11s 30ms/step - loss: 0.1393 - accuracy: 0.9482 - val_loss: 0.2778 - val_accuracy: 0.9067
Epoch 6/10
375/375 [==============================] - 12s 32ms/step - loss: 0.1338 - accuracy: 0.9498 - val_loss: 0.2968 - val_accuracy: 0.9049
Epoch 7/10
375/375 [==============================] - 13s 36ms/step - loss: 0.1247 - accuracy: 0.9529 - val_loss: 0.3135 - val_accuracy:

In [33]:
predictions = model.evaluate(val_ds)

94/94 [==============================] - 6s 59ms/step - loss: 2.2236 - accuracy: 0.1339 1s - loss: 2


In [27]:
preds = [np.argmax(p) for p in predictions]
confusion = confusion_matrix(labels = test_targets, predictions = preds, num_classes = len(np.unique(test_targets)))
fig = px.imshow(confusion, color_continuous_scale = "viridis")
fig.show()

NameError: name 'predictions' is not defined